In [3]:
import pickle

import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# Loading the data

In [4]:
with open("data/randomized/train_data", "rb") as f:
    train_data = pickle.load(f)

train_features = train_data[0]
train_targets  = train_data[1]

In [5]:
with open("data/randomized/val_data", "rb") as f:
    val_data = pickle.load(f)

val_features = val_data[0]
val_targets  = val_data[1]

In [6]:
with open("data/randomized/test_data", "rb") as f:
    test_data = pickle.load(f)

test_features = test_data[0]
test_targets  = test_data[1]

# Optimzing the Model over the validation data

In [8]:
optimizers = ["adam", "sgd"]
neurons = [pow(2, i) for i in range(5, 13)]
activaitons = ["relu", "sigmoid", "tanh"]

hps = dict()
i = 0

for opt in optimizers:
    for act in activaitons:
        for neu in neurons:
            hps[i] = {"neurons": neu, "optimizer": opt, "activation": act}
            i += 1

In [11]:
df = pd.DataFrame(columns=["accuracy", "F1-score"], index=[i for i in range(len(hps))])

for index, hp in hps.items():
    print(index)
    mlp_model = Sequential()

    mlp_model.add(layers.Input(shape = (21, 2, ), dtype = "int32"))
    mlp_model.add(layers.Flatten())
    mlp_model.add(layers.Dense(hp["neurons"], activation=hp["activation"]))
    mlp_model.add(layers.Dense(10, activation="softmax"))

    mlp_model.compile(loss="sparse_categorical_crossentropy", optimizer=hp["optimizer"], metrics=["accuracy"])

    mlp_model.fit(x=train_features, y=train_targets, epochs=10, verbose=0)

    predictions = mlp_model.predict(val_features)
    predictions = [pred.argmax() for pred in predictions]

    acc = accuracy_score(y_true=val_targets, y_pred=predictions)
    acc = round(acc*100, 2)

    f1score = f1_score(y_true=val_targets, y_pred=predictions, average="macro")
    f1score = round(f1score*100, 2)

    df["accuracy"].loc[index] = acc
    df["F1-score"].loc[index] = f1score

print(df)

print(hps)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
   accuracy F1-score
0     78.67    74.29
1     79.56    75.14
2     79.11    74.68
3     79.11    74.67
4     79.11    74.68
5     81.33    72.84
6     73.33    71.99
7     81.78     73.3
8     78.67    74.29
9     79.11    74.76
10    78.67    74.29
11    78.22    73.55
12    78.67    74.29
13    82.67    73.52
14    80.44     72.0
15    81.78    71.67
16    79.11    74.76
17    79.11    74.67
18    79.11    74.67
19    79.11    74.68
20    79.11    74.68
21    80.89    72.39
22    76.44    69.69
23    78.67    74.22
24    74.67    69.96
25    73.33     68.0
26     76.0    69.87
27    77.33    71.76
28    77.78    72.99
29    78.67     74.1
30    78.67     74.2
31    78.22    73.46
32    42.22    34.41
33    57.33    51.83
34     56.0    45.56
35    43.11    37.71
36    40.89    34.33
37    12.44     6.78
38    33.78     19.6
39    21.33     13.5
40   

In [13]:
df.sort_values(by="accuracy", ascending=False)

,accuracy,F1-score
13,82.67,73.52
7,81.78,73.3
15,81.78,71.67
5,81.33,72.84
21,80.89,72.39
14,80.44,72.0
1,79.56,75.14
16,79.11,74.76
2,79.11,74.68
3,79.11,74.67


In [14]:
print(hps[13])
print(hps[7])
print(hps[15])

{'neurons': 1024, 'optimizer': 'adam', 'activation': 'sigmoid'}
{'neurons': 4096, 'optimizer': 'adam', 'activation': 'relu'}
{'neurons': 4096, 'optimizer': 'adam', 'activation': 'sigmoid'}


# Training a baseline model

In [16]:
mlp_model = Sequential()

mlp_model.add(layers.Input(shape = (21, 2, ), dtype = "int32"))
mlp_model.add(layers.Flatten())
mlp_model.add(layers.Dense(1024, activation="sigmoid"))
mlp_model.add(layers.Dense(10, activation="softmax"))

mlp_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

baseline_history = mlp_model.fit(x=train_features, y=train_targets, epochs=10, verbose=0)

# Testing the model

In [17]:
predictions = mlp_model.predict(test_features)
predictions = [pred.argmax() for pred in predictions]

# Metrics

In [18]:
acc = accuracy_score(y_true=test_targets, y_pred=predictions)
accuracy = round(acc*100, 2)

print(f"The accuracy is {accuracy}%")

f1 = f1_score(y_true=test_targets, y_pred=predictions, average="macro")
f1 = round(f1*100, 2)

print(f"The F1-Score is {f1}%")

print(classification_report(y_true=test_targets, y_pred=predictions, digits=4))

conf_matrix = confusion_matrix(y_true=test_targets, y_pred=predictions)
px.imshow(conf_matrix, color_continuous_scale="turbo")

The accuracy is 73.87%
The F1-Score is 71.11%
              precision    recall  f1-score   support

           0     0.9725    0.9907    0.9815       107
           1     0.8182    0.4576    0.5870       118
           2     0.9821    1.0000    0.9910       110
           3     0.3633    0.8990    0.5174        99
           4     0.7128    0.5234    0.6036       128
           5     0.4940    0.4316    0.4607        95
           6     0.9864    1.0000    0.9932       145
           7     0.8667    0.1171    0.2063       111
           8     0.9914    1.0000    0.9957       115
           9     0.6594    0.9381    0.7745        97

    accuracy                         0.7387      1125
   macro avg     0.7847    0.7358    0.7111      1125
weighted avg     0.8000    0.7387    0.7218      1125

